In [1]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

## Get the Data

### Data in API

In [2]:
base_url = 'https://n6v6uai1jj.execute-api.eu-west-1.amazonaws.com/dev'
response = requests.get(base_url + '/hackathon_02/'+ '1')
print(response.text)

{"Timestamp": 1325346600, "RSI10": null, "%K200": null, "%K30": null, "EMA10": null}



In [3]:
import aiohttp
import asyncio
import time
import pandas as pd

start_time = time.time()


async def get_website(session, url, id):
    async with session.get(url) as resp:
        response = {}
        if resp.status == 200:
            response = await resp.json()
            response['id'] = id
            return response
        return None

async def main():
    responses = {}
    async with aiohttp.ClientSession() as session:

        tasks = []
        for number in range(1, 20000):
            url = f'https://n6v6uai1jj.execute-api.eu-west-1.amazonaws.com/dev/hackathon_02/{number}'
            tasks.append(asyncio.ensure_future(get_website(session, url, number)))

        websites = await asyncio.gather(*tasks)
        for website in websites:
            if website:
                responses[website['id']] = website
    
    df = pd.DataFrame.from_dict(responses, orient='index')
    df.to_csv('raw_api_data.csv', index=False)

await main()
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.977102279663086 seconds ---


In [4]:
! head raw_api_data.csv

Timestamp,RSI10,%K200,%K30,EMA10,id
1325346600,,,,,1
1325350740,,,,,2
1325350800,,,,,3
1325391360,,,,,4
1325431680,,,,,5
1325457900,,,,,6
1325534640,,,,,7
1325591100,,,,,8
1325600520,,,,4.9415769038,9


In [5]:
content = pd.read_csv('raw_api_data.csv', index_col=0)

In [6]:
content

,RSI10,%K200,%K30,EMA10,id
Timestamp,,,,,
1325346600,NaN,NaN,NaN,NaN,1
1325350740,NaN,NaN,NaN,NaN,2
1325350800,NaN,NaN,NaN,NaN,3
1325391360,NaN,NaN,NaN,NaN,4
1325431680,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...
1351989660,NaN,NaN,NaN,10.655024,19611
1351991700,NaN,NaN,NaN,10.617747,19612
1351994460,NaN,NaN,NaN,10.621793,19613


In [7]:
content.columns= content.columns.str.lower()
content = content[pd.to_numeric(content['id'], errors='coerce').notnull()]
content.index = content.index.astype(int)
content  = content.drop('id', axis =1)
content

,rsi10,%k200,%k30,ema10
Timestamp,,,,
1325346600,NaN,NaN,NaN,NaN
1325350740,NaN,NaN,NaN,NaN
1325350800,NaN,NaN,NaN,NaN
1325391360,NaN,NaN,NaN,NaN
1325431680,NaN,NaN,NaN,NaN
...,...,...,...,...
1351989660,NaN,NaN,NaN,10.655024
1351991700,NaN,NaN,NaN,10.617747
1351994460,NaN,NaN,NaN,10.621793


In [8]:
content.to_csv("api_data.csv")